In [ ]:
!pip install pynapple matplotlib dandi dandischema

# Population Decoding Tutorial 

*Authors: Dhruv Mehrotra, Guillaume Viejo*


This notebook demonstrates how we use Pynapple on various publicly available datasets in systems neuroscience to streamline analysis. In this notebook, we will examine the dataset from [Peyrache et al (2015)](https://www.nature.com/articles/nn.3968), which was used to generate Figure 4a in our <a href="https://elifesciences.org/articles/85786" target="_blank">publication</a>.  

The NWB file for the example is hosted on [OSF](https://osf.io/jb2gd). We show below how to stream it.
The entire dataset can be downloaded [here](https://dandiarchive.org/dandiset/000056).

***

**Note:** This tutorial uses matplotlib for displaying the figure as well as the dandi package

You can install all with `pip install matplotlib dandi dandischema`

***

First, import the necessary libraries:

In [ ]:
import numpy as np
import pandas as pd
import pynapple as nap
import scipy.ndimage
import matplotlib.pyplot as plt
import requests, math, os
import tqdm

***

### Downloading the data

It is a small NWB file, which we can download locally onto our system. To do so, run the following lines of code:

In [ ]:
path = "Mouse32-140822.nwb"
if path not in os.listdir("."):
    r = requests.get(f"https://osf.io/jb2gd/download", stream=True)
    block_size = 1024*1024
    with open(path, 'wb') as f:
        for data in tqdm.tqdm(r.iter_content(block_size), unit='MB', unit_scale=True,
            total=math.ceil(int(r.headers.get('content-length', 0))//block_size)):
            f.write(data)

***

### Parsing the data

Load the data and other relevant variables of interest


In [ ]:
# Load the NWB file for this dataset
data =   

# What does this look like?
print(data)

***

### Head-Direction Tuning Curves

This dataset contains units recorded from the anterior thalamus. Specifically, the anterodorsal nucleus of the thalamus (henceforth referred to as ADn), has neurons that are tuned to the direction the animal is facing in. These cells are therefore, aptly named head-direction (HD) cells. 

To study the direction-selectivity of cells in the ADn, we plot the firing rate of the neurons as a function of the animal's orientation. This plot is referred to as a HD 'tuning curve'. To plot HD tuning curves, we need the spike timings and the orientation of the animal. These quantities are stored in the variables 'units' and 'ry'.

In [ ]:
# Get spike timings
spikes = 

# Get the behavioural epochs 
epochs = 

# Get the tracked orientation of the animal
angle = 

# What does this look like ?
print(spikes)

Here, rate is the mean firing rate of the unit. Location indicates the brain region the unit was recorded from, and group refers to the shank number on which the cell was located.

As mentioned above, this dataset contains units recorded from the anterior thalamus. Head-direction (HD) cells are found only in the ADn. In this animal, units were also recorded from nearby thalamic nuclei. For the purposes of our tutorial, we are interested in the units recorded in ADn. We can restrict ourselves to analysis of these units rather easily, using Pynapple.

In [ ]:
# Select only those units that are in ADn
spikes_adn = 

# What does this look like ?
print(spikes_adn)

Let's compute some head-direction tuning curves. To do this in Pynapple, all you need is a single line of code!

Plot firing rate of ADn units as a function of heading direction, i.e. a head-direction tuning curve. 

In [ ]:
tuning_curves = 

# What does this look like ?
print(tuning_curves)

Each row indicates an angular bin (in radians), and each column corresponds to a single unit. Let's compute the preferred angle quickly as follows:

In [ ]:
pref_ang = tuning_curves.idxmax()

For easier visualization, we will colour our plots according to the preferred angle of the cell. To do so, we will normalize the range of angles we have, over a colourmap.

In [ ]:
norm = plt.Normalize()  # Normalizes data into the range [0,1]
color = plt.cm.hsv(norm([i / (2 * np.pi) for i in pref_ang.values]))  # Assigns a colour in the HSV colourmap for each value of preferred angle
color = pd.DataFrame(index=pref_ang.index, data = color, columns = ['r', 'g', 'b', 'a'])

To make the tuning curves look nice, we will smooth them before plotting, using this custom function:

In [ ]:
from scipy.ndimage import gaussian_filter1d
def smoothAngularTuningCurves(tuning_curves, sigma = 2):

    tmp = np.concatenate((tuning_curves.values, tuning_curves.values, tuning_curves.values))
    tmp = gaussian_filter1d(tmp, sigma=sigma, axis=0)

    return pd.DataFrame(index = tuning_curves.index,
        data = tmp[tuning_curves.shape[0]:tuning_curves.shape[0]*2], 
        columns = tuning_curves.columns
        )

Therefore, we have:

In [ ]:
smoothcurves = smoothAngularTuningCurves(tuning_curves, sigma = 3)

What does this look like? Let's plot the tuning curves!

In [ ]:
#Plot the tuning curves 



Awesome! 

***

### Decoding



Now that we have HD tuning curves, we can go one step further. Using only the population activity of ADn units, we can decode the direction the animal is looking in. We will then compare this to the real head direction of the animal, and discover that population activity in the ADn indeed codes for HD. 

To decode the population activity, we will be using a Bayesian Decoder as implemented in Pynapple. Just a single line of code!




In [ ]:
decoded, proba_feature = 

#What does this look like?
print(decoded)

The variable *decoded* indicates the most probable angle in which the animal was looking. 
There is another variable, *proba_feature* that denotes the probability of a given angular bin at a given time point. We can look at it below:

In [ ]:
print(proba_feature.as_dataframe())

Each row of this pandas DataFrame is a time bin, and each column is an angular bin. The sum of all values in a row add up to 1. 

Now, let's plot the raster plot for a given period of time, and overlay the actual and decoded HD on the population activity.

In [ ]:
ep =  #Select an arbitrary interval for plotting

# Plot the raster plot for each cell



# Plot the decoded HD





From this plot, we can see that the decoder is able to estimate the head-direction based on the population activity in ADn. Amazing! 

What does the probability distribution in this example event look like? \
Ideally, the bins with the highest probability will correspond to the bins having the most spikes. Let's plot the probability matrix to visualize this.

In [ ]:
# Smoothening the probability distribution
smoothed = scipy.ndimage.gaussian_filter(proba_feature, 1)  

# Create a DataFrame with the smoothed distribution
p_feature = pd.DataFrame(
    index = proba_feature.index.values,
    columns = proba_feature.columns.values,
    data = smoothed,
)

# Make it a Pynapple TsdFrame
p_feature =   

#Plot the actual HD and the decoded HD, overlaying them on the smoothed probability distribution


From this probability distribution, we observe that the decoded HD very closely matches the actual HD. Therefore, the population activity in ADn is a reliable estimate of the heading direction of the animal. 

I hope this tutorial was helpful. If you have any questions, comments or suggestions, please feel free to reach out to the Pynapple Team!